### Calculate QA scores. 
Two orthogonal measures are very effective in filtering outliers especially poor data and cloud
#### ralb 
- Relative ALBedo deviation, per pixel
#### rsad 
- Relative Spectral Angle Deviation, per pixel

#### qa_score - the 'final' variable in the dataset
qa_score is QA, normalised against a chosen threshold such acceptable values fall in the range of 0-1 (outliers are negative), where:
- QA is the combined deviation, calculated as the magnitude of the vector [rsad *w, ralb] where,
- w is an empirical weight (0.4 in practice) to permit more variation in rsad when the measures are combined in this way.
- qa_score E  x : {-inf < x < 1}.
- good data are indicated by qa_score E [0,1) 

#### DQA 
Dataset-level deviation, a fraction of pixels with rsad and ralb in reasonable range. DQA has one value for each observation time.
- nominal 'reasonable range' is set by 3n in the formulae above.

### Notes:
- the approach here has greater flexibility than taking pre-canned qa scores (and that flexibility is necessary). I suspect it is also just better, but that remains to be objectively tested.
- white clouds give very high RALB and high RSAD. Some illumination changes leave data values within normal ranges (wispy cloud, cloud shadow)
- change in the nature of the surface leads to high RSAD with normal RALB
- cloud shadow leads to low values of RALB 
- fine-tuning is possible by selecting specific filters of rsad and ralb, (for example to let more data in for ndvi calculations) but the default is pretty good. 
- a lot of experimentation to determine the thresholds used here, but the process seems quite robust and outcomes are unlikely to be changed by small changes in the parameters used in this QA process


### Outliers are detected by comparing single-observation images with the geomedian(s)
- Spectral angle divergences. SMAD is used to standardise divergences. 
- Sum of reflectances (which I loosely call 'Albedo'). BCMAD is used to standardise divergences.


In [1]:
# A function that provides a dictionary of bands to be compared with the geomedians (standard annual geomedians)
def geomedian_test_bands ():
    # --- specify the variables to compare when calculating spectral angle and albedo divergences against the geomedian
    #     Spelling out all the bands to allow contingency here for comparison of tm with oli_agm (which allows us to 
    #     extend the use of tm past 2013, esp to include L7 data   ---
    # --- the 'noIRband' version remove the IR band from the assessment. 
    #     The IR band picks up floating vegetation giving wide variations in spectral angle relative to the geomedian; these deviations are valid
    #     for wq purposes, but unusual enough to not be anticipated by the geomedian SMAD. This option allows those extremes to pass 
    #     the QA test.

    agm_tests   =   {
    'tm-v-oli_agm' : {
                        'tm'     : ['tm01'     ,'tm02'     ,'tm03'     ,'tm04'     ,'tm05'     ,'tm07'     ],
                        'oli_agm': ['oli02_agm','oli03_agm','oli04_agm','oli05_agm','oli06_agm','oli07_agm']
                    },
    'tm-v-oli_agm-noIRband' : {
                        'tm'     : ['tm01'     ,'tm02'     ,'tm03'     ,'tm05'     ,'tm07'     ],
                        'oli_agm': ['oli02_agm','oli03_agm','oli04_agm','oli06_agm','oli07_agm']
                    },
    'msi-v-msi_agm': { 
                        'msi'    : ['msi02'    ,'msi03'    ,'msi04'    ,'msi05'    ,'msi06'    ,'msi07'],
                        'msi_agm': ['msi02_agm','msi03_agm','msi04_agm','msi05_agm','msi06_agm','msi07_agm'],
                    },
    'msi-v-msi_agm-noIRband': { 
                        'msi'    : ['msi02'    ,'msi03'    ,'msi04'    ,'msi05'    ],
                        'msi_agm': ['msi02_agm','msi03_agm','msi04_agm','msi05_agm'],
                    },
    'msi-v-oli_agm-noIRband': { 
                        'msi'    : ['msi02'    ,'msi03'    ,'msi04'    ,'msi05'    ],
                        'oli_agm': ['oli02_agm','oli03_agm','oli04_agm','oli05_agm'],   #check that this is the right set. 
                    },
    'oli-v-oli_agm': { 
                        'oli'    : ['oli02'    ,'oli03'    ,'oli04'    ,'oli05'    ,'oli06'    ,'oli07'    ],
                        'oli_agm': ['oli02_agm','oli03_agm','oli04_agm','oli05_agm','oli06_agm','oli07_agm'],
                    },
    'oli-v-oli_agm-noIRband': { 
                        'oli'    : ['oli02'    ,'oli03'    ,'oli04'    ,'oli06'    ,'oli07'    ],
                        'oli_agm': ['oli02_agm','oli03_agm','oli04_agm','oli06_agm','oli07_agm'],
                    },
    'tm-v-tm_agm' : { 
                        'tm'     : ['tm01'    ,'tm02'    ,'tm03'    ,'tm04'    ,'tm05'    ,'tm07'],
                        'tm_agm' : ['tm01_agm','tm02_agm','tm03_agm','tm04_agm','tm05_agm','tm07_agm'],
                    },
    'tm-v-tm_agm-noIRband' : { 
                        'tm'     : ['tm01'    ,'tm02'    ,'tm03'    ,'tm05'    ,'tm07'],
                        'tm_agm' : ['tm01_agm','tm02_agm','tm03_agm','tm05_agm','tm07_agm'],
                    },
                }
    return(agm_tests)


#### RALB : Relative ALBedo deviation, per pixel
- RALB is calculated by summing the band values and comparing with a geomedian. The bcmad is used to normalise to 'relative' values.
- RALB values can be negative. 
-   -n  < |RALB| < n is taken as the default range of normal/typical, where n = 1.4. I.e, this is taken as equivalent to the 1-sigma range


In [2]:
# A function to calculate the relative albedo deviation

def relative_albedo(data,
                    geomedian,
                    data_band_list,
                    geomedian_band_list,
                    mad_var,
                    verbose = False,test=False): 
    
    if verbose: print("Calculating relative albedo deviations (ralb) from the geomedian ... ")
    data       = data.copy(deep=True)
    geomedian  = geomedian.copy(deep=True)
     
    # --- This now only runs on a single time-step (y,x, array) (better for memory management I hope) rather than a full time series ...
    # --- Geomedian is assumed to be a single time value (not a time-series of geomedians) 
    # --- the term 'albedo' is used loosely to mean the sum of the reflectances from relevant bands
    # --- The appropriate scale is given by the bcmad which is in the geomedian.
    
    data      = data.where(~np.isnan(data),0)
    geomedian = geomedian.where(~np.isnan(geomedian),0)

    # --- make empty arrays for the output ---
    arraysizes = data.sizes
    arrayshape = [arraysizes['y'],arraysizes['x']]
    albedo     = xr.DataArray(np.zeros((arrayshape)),dims=('y','x'))
    gm_sizes   = geomedian.sizes
    albedo_gm  = xr.DataArray(np.zeros(([gm_sizes['y'],gm_sizes['x']])),dims=('y','x'))
    
    #calculate the albedo for each pixel of the image-data and for the single-time geomedian:
    for data_var in data_band_list :
        geomedian_var = geomedian_band_list[data_band_list.index(data_var)]
        albedo        = albedo    + data[data_var].values
        albedo_gm     = albedo_gm + geomedian[geomedian_var].values #[0,:,:].values  #index here drops the single-value time coordinate

    alb_divisor       =  10000 * geomedian[mad_var] 
    relative_albedo   = (albedo - albedo_gm)/alb_divisor
    return(relative_albedo)  

#### RSAD : Relative Spectral Angle Deviation, per pixel
- RSAD is calculated as the spectral angle between the pixel and the geomedian, normalised with the smad
- RALB < n is taken as the default range of normal/typical, where n = 1.4. (I.e, this is taken as equivalent to the 1-sigma range. The literature would say 1.48 for normally distributed data; I found 1.4 from experimentation)

In [3]:
# A function to calculate the relative spectral angle deviation

def relative_spectral_angle_deviation(data,
                                      geomedian,
                                      data_band_list,
                                      geomedian_band_list,
                                      mad_var,
                                      verbose = True,test=True): 
    # Geomedian_comparison_relative_spectral_angle_deviation 
    # --- Returns pixel-by-pixel spectral angle from the geomedian,  
    #     expressed relative to the mad provided (nominally the smad). 

    if verbose: print("Calculating relative spectral angle deviations (rsad) from the geomedian... ")

    data       = data.copy(deep=True)
    geomedian  = geomedian.copy(deep=True)
    #geomedian is assumed to be a single time value (not a time-series of geomedians), but time will carry through
    # in this version the data is also assumed to be a single time-slice. This may handle memory better
    
    # --- replace nans with zeros for this process --- 
    data      = data.where(~np.isnan(data),0)
    geomedian = geomedian.where(~np.isnan(geomedian),0)
    
    # --- make some empty arrays ---
    arraysizes = data.sizes
    arrayshape = [arraysizes['y'],arraysizes['x']]
    data['dotproduct']   = ('y','x'),np.zeros(arrayshape)
    data['self_product'] = ('y','x'),np.zeros(arrayshape)
    gm_sizes   = geomedian.sizes    
    #geomedian['gm_self_product'] = ('time','y','x'),np.zeros([gm_sizes['time'],gm_sizes['y'],gm_sizes['x']])
    geomedian['gm_self_product'] = ('y','x'),np.zeros([gm_sizes['y'],gm_sizes['x']])
    
    for data_var in data_band_list : 
        geomedian_var = geomedian_band_list[data_band_list.index(data_var)]
        if test:
            print(data_var,' --- ',geomedian_var)
        
        data     ['dotproduct']      =  data['dotproduct']           +  data[data_var] * np.array(geomedian[geomedian_var]) #[0,:,:])
        data     ['self_product']    =  data['self_product']         +  data[data_var] ** 2     
        geomedian['gm_self_product'] =  geomedian['gm_self_product'] + geomedian[geomedian_var] ** 2
        
    data     ['self_product'   ] = data     ['self_product'   ] ** 0.5
    geomedian['gm_self_product'] = geomedian['gm_self_product'] ** 0.5

    # relative spectral angle deviation is the 1-cosine of the angle, divided by the smad
    data['cosdist'] =     data['dotproduct'] / (data['self_product'] * np.array(geomedian['gm_self_product']))  #[0,:,:]))
    data['sad']     =     1 - data['cosdist'] 
    data['rsad']    =     data['sad'] / np.array(geomedian[mad_var] )  #[0,:,:] )
        
    return(data['rsad'])
    